This notebook finds Rogue Scholar blog posts published the last seven days using the [Rogue Scholar API](https://api.rogue-scholar.org/posts).

:::{.callout-note}
* We limit results to posts published since `2023-11-08` (i.e. last week) and `en` as language.
* We retrieve the `title`, `authors`, `publication date`, `abstract`, `blog name`, `blog_slug`, and `doi`.
* We sort the results in chronological order (oldest first).
* We format the results and generate markdown output.
:::

In [4]:
# | label: query

import requests
import locale
import html
import json
import re
from typing import Optional
import datetime
import pydash as py_
from IPython.display import Markdown

locale.setlocale(locale.LC_ALL, "en_US")
baseUrl = "https://api.rogue-scholar.org/"
published_since = "2023-11-08"
published_until = "2023-11-14"
feature_image = 2
include_fields = "title,authors,published_at,summary,blog_name,blog_slug,doi,url,image"
url = (
    baseUrl
    + f"posts?&published_since={published_since}&published_until={published_until}&language=en&sort=published_at&order=asc&per_page=50&include_fields={include_fields}"
)
response = requests.get(url)
result = response.json()


def get_post(post):
    return post["document"]


def format_post(post):
    doi = post.get("doi", None)
    url = f"[{doi}]({doi})\n<br />" if doi else ""
    title = f"[{post['title']}]({doi})" if doi else f"[{post['title']}]({post['url']})"
    published_at = datetime.datetime.utcfromtimestamp(post["published_at"]).strftime(
        "%B %-d, %Y"
    )
    blog = f"[{post['blog_name']}](https://rogue-scholar.org/blogs/{post['blog_slug']})"
    author = ", ".join([f"{x['name']}" for x in post.get("authors", None) or []])
    summary = post["summary"]
    return f"### {title}\n{url}Published {published_at} in {blog}<br />{author}<br /><br />{summary}\n"


posts = [get_post(x) for i, x in enumerate(result["hits"])]
posts_as_string = "\n".join([format_post(x) for x in posts])

def doi_from_url(url: str) -> Optional[str]:
    """Return a DOI from a URL"""
    match = re.search(
        r"\A(?:(http|https)://(dx\.)?(doi\.org|handle\.stage\.datacite\.org|handle\.test\.datacite\.org)/)?(doi:)?(10\.\d{4,5}/.+)\Z",
        url,
    )
    if match is None:
        return None
    return match.group(5).lower()

# Get csl-formatted metadata for all posts that have a DOI
def get_csl(post):
    doi = doi_from_url(post["doi"])
    res = requests.get(baseUrl + "posts/" + doi + "?format=csl")
    csl = res.json()
    
    # remove keys not compatible with CSL spec
    csl = py_.omit(csl, ["license", "original-title"])
    csl["id"] = post["doi"]
    csl["title"] = html.unescape(post["title"])
    csl["container-title"] = post["blog_name"]
    csl["type"] = "article-journal"
    return json.dumps(csl, indent=2)

csl_list = "[\n" + ",\n".join([ get_csl(x) for x in posts if x.get("doi", None) is not None ]) + "\n]"
with open('digest.json', 'w') as f:
    f.write(csl_list)

images = [x["image"] for x in posts if x.get("image", None) is not None]
image = images[feature_image]
markdown = f"![]({image})\n\n"
markdown += posts_as_string
Markdown(markdown)

![](https://www.scholcommlab.ca/wp-content/uploads/2023/11/STI2023-group-photo.jpg)

### [Introducing the CWTS ECR/PhD Council](https://www.leidenmadtrics.nl/articles/introducing-the-cwts-ecr-phd-council)
Published November 8, 2023 in [Leiden Madtrics](https://rogue-scholar.org/blogs/leidenmadtrics)<br />Leiden Madtrics<br /><br />In this blog post, we'll introduce you to the CWTS ECR/PhD Council, shedding light on its significance, objectives, and how it can serve as a crucial resource for early career researchers and PhDs at CWTS. CWTS is an interdisciplinary institute at Leiden University with a diverse multinational academic culture. PhD candidates, early career researchers, and visiting scholars from around the world gather at CWTS to pursue their research journeys.

### [BioBanking at GGBN 2023: Highlights from GigaScience Press](https://doi.org/10.59350/ahnf6-e1m90)
[https://doi.org/10.59350/ahnf6-e1m90](https://doi.org/10.59350/ahnf6-e1m90)
<br />Published November 8, 2023 in [GigaBlog](https://rogue-scholar.org/blogs/gigablog)<br />Chris Armit<br /><br /><em>
 “Extinction is forever – so our action must be immediate.”
</em>
–
<em>
 Sir David Attenborough, Sept 30th 2020
</em>
The fourth international Global Genome Biodiversity Network (GGBN) Conference took place in Aguascalientes, Mexico from October 17th to October 20th 2023 where it was hosted by the Universidad Autónoma de Aguascalientes.


### [From Ottawa to Leiden: ScholCommLab&#8217;s Conference Highlights](https://doi.org/10.59350/vbp1j-f9j63)
[https://doi.org/10.59350/vbp1j-f9j63](https://doi.org/10.59350/vbp1j-f9j63)
<br />Published November 9, 2023 in [Scholarly Communications Lab | ScholCommLab](https://rogue-scholar.org/blogs/scholcommlab)<br />Olivia Aguiar<br /><br />Over the last few months, ScholCommLab members took part in over six conferences, including the: International Conference on Science, Technology, and Innovation Indicators (STI 2023), International Communication Association Conference (ICA 2023), and Bibliometrics and Research Impact Community Conference (BRIC 2023).  In this blog post, we’re sharing highlights from each event.


### [Archiving Rogue Scholar blogs with the Internet Archive](https://doi.org/10.53731/g60vh-3ng48)
[https://doi.org/10.53731/g60vh-3ng48](https://doi.org/10.53731/g60vh-3ng48)
<br />Published November 9, 2023 in [Front Matter](https://rogue-scholar.org/blogs/front_matter)<br />Martin Fenner<br /><br />Blogs participating in the Rogue Scholar science blog archive are now archived in the Internet Archive. Starting November 1st, Rogue Scholar is participating in the Internet Archive Archive-It service and all archived blogs can be found here.

### [Two influential textbooks &#8211; &#8220;Mee&#8221;  and &#8220;Mellor&#8221;.](https://doi.org/10.59350/e3wmw-qwx29)
[https://doi.org/10.59350/e3wmw-qwx29](https://doi.org/10.59350/e3wmw-qwx29)
<br />Published November 11, 2023 in [Henry Rzepa's Blog](https://rogue-scholar.org/blogs/rzepa)<br />Henry Rzepa<br /><br />I am a member of the &nbsp;Royal Society of &nbsp;Chemistry’s Historical group. Amongst other activities, it publishes two editions&nbsp;of a newsletter each year for its members.

### [Auto-DOI for Quarto posts via Rogue Scholar](https://doi.org/10.59350/5hxdg-fz574)
[https://doi.org/10.59350/5hxdg-fz574](https://doi.org/10.59350/5hxdg-fz574)
<br />Published November 13, 2023 in [Chris von Csefalvay](https://rogue-scholar.org/blogs/chrisvoncsefalvay)<br />Chris von Csefalvay<br /><br />I love posts that allow me to merge some of my addictions.

### [Data on 100 million individual trees in the National Ecological Observatory Network](https://doi.org/10.59350/6vjpn-ky077)
[https://doi.org/10.59350/6vjpn-ky077](https://doi.org/10.59350/6vjpn-ky077)
<br />Published November 13, 2023 in [Jabberwocky Ecology](https://rogue-scholar.org/blogs/jabberwocky_ecology)<br />Ethan White<br /><br />We’re excited to announce the initial release of crown maps for 100 million trees in the National Ecological Observatory Network (NEON) with information on location, species identify, size, and alive/dead status.

### [Do you dare? What female scientists expect when communicating](https://elephantinthelab.org/do-you-dare-what-female-scientists-expect-when-communicating)
Published November 14, 2023 in [Elephant in the Lab](https://rogue-scholar.org/blogs/elephantinthelab)<br />Sascha Schönig<br /><br /><strong> The expectation to communicate</strong>   In 2019, the German Federal Ministry of Education and Research called for a cultural shift toward communicating science (Bundesministerium für Bildung und Forschung, 2019). The former Federal Minister of Education and Research, Anja Karliczek, urged scientists to communicate their research continuously and classified communication as a central task for universities and research organizations

### [Reevaluating POSI: 2023 evolution](https://doi.org/10.59350/z537h-xsd22)
[https://doi.org/10.59350/z537h-xsd22](https://doi.org/10.59350/z537h-xsd22)
<br />Published November 14, 2023 in [Liberate Science](https://rogue-scholar.org/blogs/libscie)<br />Liberate Science<br /><br />In 2022, we did our first self-evaluation according to the Principles of Open Scholarly Infrastructure (POSI). In this blog, we take stock ~1 year later, based on POSI v1.1, and reevaluate how we're doing and where there's still room for improvement. Summary  💚 = aligned; 💛 = less aligned;

### [Upstream reflections](https://doi.org/10.59350/qxhxa-kac46)
[https://doi.org/10.59350/qxhxa-kac46](https://doi.org/10.59350/qxhxa-kac46)
<br />Published November 14, 2023 in [Chris Hartgerink](https://rogue-scholar.org/blogs/chjh)<br />Chris Hartgerink<br /><br />Recently, I left the Upstream editorial team as a result of changing priorities.&nbsp; I figured it would be a good idea to reflect before I forget.

### [New paper: theropod bite marks on Morrison sauropod bones](https://doi.org/10.59350/txkht-ewp95)
[https://doi.org/10.59350/txkht-ewp95](https://doi.org/10.59350/txkht-ewp95)
<br />Published November 14, 2023 in [Sauropod Vertebra Picture of the Week](https://rogue-scholar.org/blogs/svpow)<br />Matt Wedel<br /><br />&nbsp; New paper out today in PeerJ: Lei R, Tschopp E, Hendrickx C, Wedel MJ, Norell M, Hone DWE. 2023. Bite and tooth marks on sauropod dinosaurs from the Morrison Formation. PeerJ 11:e16327 http://doi.org/10.7717/peerj.16327 This one had a long gestation.

### [The programmable blog](https://doi.org/10.53731/ch2e1-1cm64)
[https://doi.org/10.53731/ch2e1-1cm64](https://doi.org/10.53731/ch2e1-1cm64)
<br />Published November 14, 2023 in [Front Matter](https://rogue-scholar.org/blogs/front_matter)<br />Martin Fenner<br /><br />Today I am relaunching the Syldavia Gazette blog on a new blogging platform, switching from Ghost to Quarto. This allows me to use Jupyter notebooks in the blog to help generate blog posts. The Syldavia Gazette (the other blog I manage besides the Front Matter blog) description says The Syldavia Gazette is a newsletter highlighting interesting science stories from around the web.


## References

::: {#refs}
:::